# Drone Video ➜ Web‑Ready Mesh & Gaussian Splats (GPU + Xvfb)
Complete Colab pipeline with GPU SIFT & Patch‑Match enabled via a virtual X‑server.

## 1 – Mount Drive & set paths

In [ ]:
from google.colab import drive, files
import os, time

drive.mount('/content/drive')

PROJECT_DIR = "/content/drive/MyDrive/Colmap"
IMAGES_DIR  = f"{PROJECT_DIR}/images"
OUTPUTS_DIR = f"{PROJECT_DIR}/outputs"
os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(OUTPUTS_DIR, exist_ok=True)
print(PROJECT_DIR)

MessageError: Error: credential propagation was unsuccessful

## 2 – Install COLMAP, ffmpeg, Open3D, Xvfb

In [ ]:
!sudo apt-get update -qq
!sudo apt-get install -y colmap ffmpeg xvfb
!pip install -q open3d>=0.17.0

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.14).
The following additional packages will be installed:
  libamd2 libcamd2 libccolamd2 libceres2 libcholmod3 libcolamd2 libcxsparse3
  libevdev2 libfreeimage3 libgflags2.2 libgoogle-glog0v5 libgudev-1.0-0
  libinput-bin libinput10 libjxr0 libmd4c0 libmetis5 libmtdev1 libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libraw20
  libspqr2 libsuitesparseconfig5 libwacom-bin libwacom-common libwacom9
  libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-render-util0 libxcb-util1
  libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1 libxkbcommon-x11-0
  qt5-gtk-pla

## 3 – Extract frames

In [ ]:
VIDEO_PATH="/content/drive/MyDrive/your_video.mp4"  # <- change
FRAME_RATE=2.0  # 0 = all
fps_filter=f"-vf fps={FRAME_RATE}" if FRAME_RATE and FRAME_RATE>0 else ""
!ffmpeg -i "{VIDEO_PATH}" {fps_filter} -qscale:v 2 "{IMAGES_DIR}/frame_%04d.jpg"
print('frames ready')

## 4 – Run COLMAP in Xvfb

In [ ]:
import os, time, textwrap
os.environ["QT_QPA_PLATFORM"]="offscreen"
log=f"{PROJECT_DIR}/logs"; os.makedirs(log, exist_ok=True)
log_file=f"{log}/auto_{time.strftime('%Y%m%d_%H%M%S')}.log"
cmd=textwrap.dedent(f'''
xvfb-run --auto-servernum --server-args="-screen 0 1024x768x24" \
  colmap automatic_reconstructor \
    --workspace_path "{PROJECT_DIR}" \
    --image_path "{IMAGES_DIR}" \
    --quality high \
    --dense 1 \
    --num_threads 2 \
    --use_gpu 1
''')
!bash -c '{cmd}' | tee "{log_file}"


QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'

Feature extraction

Processed file [1/109]
  Name:            frame_0001.jpg
  Dimensions:      3840 x 2160
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    4608.00px
  Features:        10116
Processed file [2/109]
  Name:            frame_0002.jpg
  Dimensions:      3840 x 2160
  Camera:          #2 - SIMPLE_RADIAL
  Focal Length:    4608.00px
  Features:        9868
Processed file [3/109]
  Name:            frame_0003.jpg
  Dimensions:      3840 x 2160
  Camera:          #3 - SIMPLE_RADIAL
  Focal Length:    4608.00px
  Features:        10090
Processed file [4/109]
  Name:            frame_0004.jpg
  Dimensions:      3840 x 2160
  Camera:          #4 - SIMPLE_RADIAL
  Focal Length:    4608.00px
  Features:        10060
Processed file [5/109]
  Name:            frame_0005.jpg
  Dimensions:      3840 x 2160
  Camera:          #5 - SIMPLE_RADIAL
  Focal Length:    4608.00px
  Features:        10084
Proce

In [ ]:
# ⚡️ Sequential SIFT matching  (k = 2) -----------------------------------
# Pre-req: feature_extractor has filled `database.db`.

import os, subprocess, sys, time, textwrap
os.environ["QT_QPA_PLATFORM"] = "offscreen"   # head-less Qt for Xvfb

DB_PATH = f"{PROJECT_DIR}/database.db"
LOG_DIR = f"{PROJECT_DIR}/logs"; os.makedirs(LOG_DIR, exist_ok=True)
LOG     = f"{LOG_DIR}/seq_match_k2_{time.strftime('%Y%m%d_%H%M%S')}.log"

cmd = textwrap.dedent(f"""
xvfb-run --auto-servernum --server-args="-screen 0 800x600x24" \\
  colmap sequential_matcher \\
    --database_path "{DB_PATH}" \\
    --SequentialMatching.overlap 2 \\
    --SiftMatching.use_gpu 1
""").strip()

print("🚀  Running sequential matcher (overlap = 2)...")
rv = subprocess.call(cmd, shell=True)         # runs & streams output
if rv != 0:
    sys.exit("❌ sequential_matcher failed")

print("✅  Sequential matching finished.")
print("📄  Log saved to", LOG)



🚀  Running sequential matcher (overlap = 2)...


KeyboardInterrupt: 

In [ ]:
# mount vars first : PROJECT_DIR, etc.

!xvfb-run --auto-servernum --server-args="-screen 0 640x480x24" \
  colmap sequential_matcher \
    --database_path "{PROJECT_DIR}/database.db" \
    --SequentialMatching.overlap 2 \
    --SiftMatching.use_gpu 1 \
    --SiftMatching.gpu_index 0 \
    --SiftMatching.guided_matching 1



QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'

Sequential feature matching

Matching image [1/109] in 0.000s
Matching image [2/109] in 0.000s
Matching image [3/109] in 0.000s
Matching image [4/109] in 0.000s
Matching image [5/109] in 0.000s
Matching image [6/109] in 0.000s
Matching image [7/109] in 0.000s
Matching image [8/109] in 0.000s
Matching image [9/109] in 0.000s
Matching image [10/109] in 0.000s
Matching image [11/109] in 56.550s
Matching image [12/109] in 55.378s
Matching image [13/109] in 52.093s
Matching image [14/109] in 52.468s
Matching image [15/109] in 52.584s
Matching image [16/109] in 51.546s
Matching image [17/109] in 50.644s
Matching image [18/109] in 49.270s
Matching image [19/109] in 49.385s
Matching image [20/109] in 47.135s
Matching image [21/109] in 45.635s
Matching image [22/109] in 44.649s
Matching image [23/109] in 43.925s
Matching image [24/109] in 42.212s
Matching image [25/109] in 42.687s
Matching image [26/109] in 50.155s
Matc

In [ ]:
import os
os.makedirs(f"{PROJECT_DIR}/sparse", exist_ok=True)   # <-- create the dir first



In [ ]:
!xvfb-run -a colmap mapper \
    --database_path "{PROJECT_DIR}/database.db" \
    --image_path    "{IMAGES_DIR}" \
    --output_path   "{PROJECT_DIR}/sparse" \
    --Mapper.ba_global_images_freq 1


Streaming output truncated to the last 5000 lines.
  => Completed observations: 13
  => Merged observations: 8
  => Retriangulated observations: 0

Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.946841e+05    0.00e+00    1.78e+04   0.00e+00   0.00e+00  1.00e+04        0    8.36e-01    4.14e+00

CHOLMOD version 3.0.14, Oct 22, 2019: Symbolic Analysis: status: OK
  Architecture: Linux
    sizeof(int):      4
    sizeof(SuiteSparse_long):  8
    sizeof(void *):   8
    sizeof(double):   8
    sizeof(Int):      4 (CHOLMOD's basic integer)
    sizeof(BLAS_INT): 4 (integer used in the BLAS)
  Results from most recent analysis:
    Cholesky flop count: 4.5134e+07
    Nonzeros in L:       1.3184e+05
  memory blocks in use:          11
  memory in use (MB):           0.0
  peak memory usage (MB):       0.5
  maxrank:    update/downdate rank:   8
  supernodal control: 1 40 (supernodal if flops/lnz >= 4

In [ ]:
!tail -n 40 "{PROJECT_DIR}/logs/mapper.log"   # or whatever log file you tee’d


tail: cannot open '/content/drive/MyDrive/Drone3DProject/logs/mapper.log' for reading: No such file or directory


In [ ]:
# 🚀 Dense reconstruction  (undistort ➜ depth ➜ fusion)
# Assumes PROJECT_DIR, IMAGES_DIR, SPARSE_DIR are already defined.

import os, subprocess, textwrap, sys, time, pathlib

os.environ["QT_QPA_PLATFORM"] = "offscreen"   # head-less Qt

DENSE_DIR = f"{PROJECT_DIR}/dense"
pathlib.Path(DENSE_DIR).mkdir(parents=True, exist_ok=True)

cmd = textwrap.dedent(f"""
  # 1️⃣  Undistort images  (2500 px long side)
  xvfb-run -a colmap image_undistorter \
      --image_path "{IMAGES_DIR}" \
      --input_path "{SPARSE_DIR}/0" \
      --output_path "{DENSE_DIR}" \
      --max_image_size 2500 &&

  # 2️⃣  Patch-Match Stereo  (fewer samples, GPU 0)
  xvfb-run -a colmap patch_match_stereo \
      --workspace_path "{DENSE_DIR}" \
      --PatchMatchStereo.geom_consistency true \
      --PatchMatchStereo.num_samples 11 \
      --PatchMatchStereo.filter false \
      --gpu_index 0 &&

  # 3️⃣  Stereo Fusion  (lower RAM spike)
  xvfb-run -a colmap stereo_fusion \
      --workspace_path "{DENSE_DIR}" \
      --output_path "{DENSE_DIR}/fused.ply" \
      --min_num_pixels 4
""").strip()

print("🟢  Running dense pipeline ...")
ret = subprocess.call(cmd, shell=True)
if ret != 0:
    sys.exit("❌  Dense reconstruction failed — check console for the exact error.")
else:
    print("\n✅  fused.ply written to", f"{DENSE_DIR}/fused.ply")


🟢  Running dense pipeline ...


SystemExit: ❌  Dense reconstruction failed — check console for the exact error.

In [ ]:
%%bash
set -eux  # stop on first error, echo commands

# 0. Always begin in a valid directory
cd /content

# 1. Remove any previous clone
rm -rf colmap

# 2. Install build deps (takes <30 s if already cached)
sudo apt-get update -qq
sudo apt-get install -y --no-install-recommends \
  git cmake ninja-build build-essential \
  libboost-program-options-dev libboost-system-dev libboost-filesystem-dev \
  libeigen3-dev libflann-dev libfreeimage-dev libsqlite3-dev \
  libgoogle-glog-dev libgflags-dev libglew-dev qtbase5-dev libqt5opengl5-dev \
  libcgal-dev libcgal-qt5-dev libceres-dev nvidia-cuda-toolkit

# 3. Fresh clone with sub-modules
git clone --recursive https://github.com/colmap/colmap.git
cd colmap

# 4. Configure in a new build/ folder – CUDA ON, GUI OFF, arch=75 (Tesla T4)
mkdir build && cd build
cmake .. -GNinja \
  -DCMAKE_BUILD_TYPE=Release \
  -DCUDA_ENABLED=ON \
  -DGUI_ENABLED=OFF \
  -DCMAKE_CUDA_ARCHITECTURES=75    # ← manual arch, avoids “native” probe

# 5. Compile & install (≈15 min with 2 threads)
ninja -j2
sudo ninja install


Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.9ubuntu3).
libboost-filesystem-dev is already the newest version (1.74.0.3ubuntu7).
libboost-program-options-dev is already the newest version (1.74.0.3ubuntu7).
libboost-system-dev is already the newest version (1.74.0.3ubuntu7).
libceres-dev is already the newest version (2.0.0+dfsg1-5).
libcgal-dev is already the newest version (5.4-1).
libcgal-qt5-dev is already the newest version (5.4-1).
libeigen3-dev is already the newest version (3.4.0-2ubuntu2).
libflann-dev is already the newest version (1.9.1+dfsg-11).
libgflags-dev is already the newest version (2.2.2-2).
libglew-dev is already the newest version (2.2.0-4).
libgoogle-glog-dev is already the newest version (0.5.0+really0.4.0-2).
ninja-build is already the newest version (1.10.1-1).
nvidia-cuda-toolkit is already the newest version (11.5.1-1ubuntu1).
cmake is already the newest version (3.22.1-1u

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
+ cd /content
chdir: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
+ rm -rf colmap
+ sudo apt-get update -qq
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
+ sudo apt-get install -y --no-install-recommends git cmake ninja-build build-essential libboost-program-options-dev libboost-system-dev libboost-filesystem-dev libeigen3-dev libflann-dev libfreeimage-dev libsqlite3-dev libgoogle-glog-dev libgflags-dev libglew-dev qtbase5-dev libqt5opengl5-dev libcgal-dev libcgal-qt5-dev libceres-dev nvidia-cuda-toolkit
+ git clone --recursive https://github.com/colmap/colmap.git
Cloning into 'colmap'...
Submodule 'doc/_build/html' (https://github.com/colmap/colmap.github.io.git) registere

CalledProcessError: Command 'b'set -eux  # stop on first error, echo commands\n\n# 0. Always begin in a valid directory\ncd /content\n\n# 1. Remove any previous clone\nrm -rf colmap\n\n# 2. Install build deps (takes <30 s if already cached)\nsudo apt-get update -qq\nsudo apt-get install -y --no-install-recommends \\\n  git cmake ninja-build build-essential \\\n  libboost-program-options-dev libboost-system-dev libboost-filesystem-dev \\\n  libeigen3-dev libflann-dev libfreeimage-dev libsqlite3-dev \\\n  libgoogle-glog-dev libgflags-dev libglew-dev qtbase5-dev libqt5opengl5-dev \\\n  libcgal-dev libcgal-qt5-dev libceres-dev nvidia-cuda-toolkit\n\n# 3. Fresh clone with sub-modules\ngit clone --recursive https://github.com/colmap/colmap.git\ncd colmap\n\n# 4. Configure in a new build/ folder \xe2\x80\x93 CUDA ON, GUI OFF, arch=75 (Tesla T4)\nmkdir build && cd build\ncmake .. -GNinja \\\n  -DCMAKE_BUILD_TYPE=Release \\\n  -DCUDA_ENABLED=ON \\\n  -DGUI_ENABLED=OFF \\\n  -DCMAKE_CUDA_ARCHITE

In [ ]:
!sudo apt-get update -qq
!sudo apt-get install -y libmetis-dev libsuitesparse-dev

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libmetis-dev is already the newest version (5.1.0.dfsg-7build2).
libsuitesparse-dev is already the newest version (1:5.10.1+dfsg-4build1).
libsuitesparse-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.


In [ ]:
# NEW cell – make sure we’re inside the build directory you created earlier
%cd /content/colmap/build

!cmake .. -GNinja \
    -DCMAKE_BUILD_TYPE=Release \
    -DCUDA_ENABLED=ON \
    -DGUI_ENABLED=OFF \
    -DCMAKE_CUDA_ARCHITECTURES=75     # T4 = sm_75

!ninja -j2
!sudo ninja install


/content/colmap/build
-- Enabling LSD support
-- Found OpenMP_C: -fopenmp (found version "4.5")
-- Found OpenMP_CXX: -fopenmp (found version "4.5")
-- Found OpenMP: TRUE (found version "4.5")
-- Found FreeImage
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/libfreeimage.so
-- Found Metis
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/libmetis.so
-- Found Glog
--   Target : glog::glog
-- Found Glew
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/libGLEW.so
-- Found required Ceres dependency: Eigen version 3.4.0 in /usr/include/eigen3
-- Found required Ceres dependency: glog
-- Found required Ceres dependency: gflags
-- Found Ceres version: 2.0.0 installed in: /usr with components: [EigenSparse, SparseLinearAlgebraLibrary, LAPACK, SuiteSparse, CXSparse, SchurSpecializations, Multithreading]
-- Using header-only CGAL
-- Targetting Ninja
-- Using /usr/bin/c++ compiler.
-- Boost include dirs: /usr/include
-- Boost lib

In [ ]:
# ▶️ Mount Drive (if not mounted already)
from google.colab import drive, files
import os, pathlib
drive.mount('/content/drive')

PROJECT_DIR = "/content/drive/MyDrive/Drone3DProject"
IMAGES_DIR  = f"{PROJECT_DIR}/images"
SPARSE_DIR  = f"{PROJECT_DIR}/sparse"
DENSE_DIR   = f"{PROJECT_DIR}/dense"
os.makedirs(DENSE_DIR,  exist_ok=True)

print("Project:", PROJECT_DIR)


Mounted at /content/drive
Project: /content/drive/MyDrive/Drone3DProject


In [ ]:
LOG = f"{PROJECT_DIR}/logs/undistorter.log"
!xvfb-run -a colmap image_undistorter \
    --image_path "{IMAGES_DIR}" \
    --input_path "{SPARSE_DIR}/0" \
    --output_path "{DENSE_DIR}" \
    --max_image_size 2500  2>&1 | tee "{LOG}"

!tail -n 20 "{LOG}"


I0609 09:52:17.583580 35189 misc.cc:44] 
Reading reconstruction
I0609 09:52:23.404419 35189 image.cc:361] => Reconstruction with 109 images and 62028 points
I0609 09:52:23.404579 35189 misc.cc:44] 
Image undistortion
I0609 09:52:23.763978 35189 undistortion.cc:197] Undistorting image [1/109]
I0609 09:52:30.075379 35189 undistortion.cc:197] Undistorting image [2/109]
I0609 09:52:30.076789 35189 undistortion.cc:197] Undistorting image [3/109]
I0609 09:52:37.047571 35189 undistortion.cc:197] Undistorting image [4/109]
I0609 09:52:37.050246 35189 undistortion.cc:197] Undistorting image [5/109]
I0609 09:52:43.189620 35189 undistortion.cc:197] Undistorting image [6/109]
I0609 09:52:43.199774 35189 undistortion.cc:197] Undistorting image [7/109]
I0609 09:52:46.845063 35189 undistortion.cc:197] Undistorting image [8/109]
I0609 09:52:47.979423 35189 undistortion.cc:197] Undistorting image [9/109]
I0609 09:52:50.829516 35189 undistortion.cc:197] Undistorting image [10/109]
I0609 09:52:53.865125 

In [ ]:
!colmap -h | head -n 3          # should show “COLMAP … (CUDA)”
!nvidia-smi --query-gpu=name,memory.total --format=csv


COLMAP 3.12.0.dev0 -- Structure-from-Motion and Multi-View Stereo
(Commit 3e3ecb16 on 2025-06-06 with CUDA)

name, memory.total [MiB]
Tesla T4, 15360 MiB


In [ ]:
# Patch-Match Stereo (GPU)  — one-liner with !
LOG = f"{PROJECT_DIR}/logs/patchmatch_gpu.log"

!xvfb-run -a colmap patch_match_stereo \
    --workspace_path "{DENSE_DIR}" \
    --PatchMatchStereo.geom_consistency true \
    --PatchMatchStereo.num_samples 11 \
    --PatchMatchStereo.filter false \
    --PatchMatchStereo.gpu_index 0  2>&1 | tee "$LOG"

!tail -n 20 "$LOG"


Streaming output truncated to the last 5000 lines.
I0609 11:03:18.203898 38150 patch_match_options.cc:51] window_radius: 5
I0609 11:03:18.203900 38150 patch_match_options.cc:52] window_step: 1
I0609 11:03:18.203902 38150 patch_match_options.cc:53] sigma_spatial: 5
I0609 11:03:18.203905 38150 patch_match_options.cc:54] sigma_color: 0.2
I0609 11:03:18.203908 38150 patch_match_options.cc:55] num_samples: 11
I0609 11:03:18.203910 38150 patch_match_options.cc:56] ncc_sigma: 0.6
I0609 11:03:18.203913 38150 patch_match_options.cc:57] min_triangulation_angle: 1
I0609 11:03:18.203917 38150 patch_match_options.cc:58] incident_angle_sigma: 0.9
I0609 11:03:18.203919 38150 patch_match_options.cc:59] num_iterations: 5
I0609 11:03:18.203922 38150 patch_match_options.cc:60] geom_consistency: 1
I0609 11:03:18.203924 38150 patch_match_options.cc:61] geom_consistency_regularizer: 0.3
I0609 11:03:18.203926 38150 patch_match_options.cc:62] geom_consistency_max_cost: 3
I0609 11:03:18.203929 38150 patch_matc

In [ ]:
!nvidia-smi


Mon Jun  9 08:27:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# 1️⃣  Stereo Fusion  – create fused.ply  (no min_num_pixels flag)
LOG_FUSION = f"{PROJECT_DIR}/logs/fusion_gpu.log"
!xvfb-run -a colmap stereo_fusion \
    --workspace_path "{DENSE_DIR}" \
    --output_path    "{DENSE_DIR}/fused.ply"  2>&1 | tee "$LOG_FUSION"

print("\nFusion log tail:")
!tail -n 20 "$LOG_FUSION"

# 2️⃣  Poisson Mesher  – surface reconstruction (threads auto-chosen)
LOG_MESH = f"{PROJECT_DIR}/logs/poisson_gpu.log"
!xvfb-run -a colmap poisson_mesher \
    --input_path  "{DENSE_DIR}/fused.ply" \
    --output_path "{DENSE_DIR}/meshed-poisson.ply"  2>&1 | tee "$LOG_MESH"

print("\nMesher log tail:")
!tail -n 20 "$LOG_MESH"

print("\n✅  Outputs (on Drive):")
print("   Dense cloud :", f"{DENSE_DIR}/fused.ply")
print("   Poisson mesh:", f"{DENSE_DIR}/meshed-poisson.ply")


I0609 11:47:19.096218 64687 misc.cc:51] 
StereoFusion::Options
---------------------
I0609 11:47:19.096470 64687 fusion.cc:76] mask_path: 
I0609 11:47:19.096529 64687 fusion.cc:77] max_image_size: -1
I0609 11:47:19.096575 64687 fusion.cc:78] min_num_pixels: 5
I0609 11:47:19.096629 64687 fusion.cc:79] max_num_pixels: 10000
I0609 11:47:19.096673 64687 fusion.cc:80] max_traversal_depth: 100
I0609 11:47:19.096719 64687 fusion.cc:81] max_reproj_error: 2
I0609 11:47:19.096779 64687 fusion.cc:82] max_depth_error: 0.01
I0609 11:47:19.096827 64687 fusion.cc:83] max_normal_error: 10
I0609 11:47:19.096875 64687 fusion.cc:84] check_num_images: 50
I0609 11:47:19.096922 64687 fusion.cc:85] use_cache: 0
I0609 11:47:19.096967 64687 fusion.cc:86] cache_size: 32
I0609 11:47:19.097013 64687 fusion.cc:89] bbox_min: -3.40282e+38 -3.40282e+38 -3.40282e+38
I0609 11:47:19.097074 64687 fusion.cc:90] bbox_max: 3.40282e+38 3.40282e+38 3.40282e+38
I0609 11:47:19.097147 64687 fusion.cc:140] Reading workspace...
I0

## 5 – Clean point cloud & mesh

In [ ]:
## 🏗️  Poisson reconstruction from fused.ply  ----------------------------
# Assumes fused.ply is at  /content/drive/MyDrive/Drone3DProject/dense/fused.ply
# 🏗️  Poisson reconstruction with Open3D (GPU not required)
!pip install -q open3d>=0.17.0

import open3d as o3d, numpy as np, os, pathlib, sys

# ---- paths ------------------------------------------------------------
PROJECT_DIR = "/content/drive/MyDrive/Drone3DProject"
DENSE_DIR   = f"{PROJECT_DIR}/dense"
OUTPUTS_DIR = f"{PROJECT_DIR}/outputs"; pathlib.Path(OUTPUTS_DIR).mkdir(exist_ok=True)

pc_path   = f"{DENSE_DIR}/fused.ply"               # input point-cloud
mesh_ply  = f"{OUTPUTS_DIR}/mesh_poisson_o3d.ply"   # cleaned mesh
mesh_glb  = f"{OUTPUTS_DIR}/mesh_poisson_o3d.glb"

if not os.path.exists(pc_path):
    sys.exit(f"❌ {pc_path} not found – run Stereo Fusion first.")

# ---- load & estimate normals -----------------------------------------
pcd = o3d.io.read_point_cloud(pc_path)
print("Points in cloud:", len(pcd.points))

pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
    radius=0.03, max_nn=30))        # radius ~ 3 cm in world units; tweak if needed
pcd.orient_normals_consistent_tangent_plane(k=10)

# ---- Poisson reconstruction ------------------------------------------
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
    pcd, depth=11, width=0, scale=1.1, linear_fit=False)
print("Mesh vertices:", len(mesh.vertices), "triangles:", len(mesh.triangles))

# ---- remove low-density vertices (gets rid of loose blobs) ------------
densities = np.asarray(densities)
density_thresh = np.percentile(densities, 5)        # drop lowest 5 %
verts_to_keep  = densities > density_thresh
mesh = mesh.select_by_index(np.where(verts_to_keep)[0])

mesh.remove_unreferenced_vertices()
mesh.remove_degenerate_triangles()
mesh.remove_duplicated_triangles()
mesh.remove_non_manifold_edges()
mesh.compute_vertex_normals()

# ---- write outputs ----------------------------------------------------
o3d.io.write_triangle_mesh(mesh_ply, mesh)
o3d.io.write_triangle_mesh(mesh_glb, mesh)

print("\n✅  Poisson mesh saved:")
print("   PLY :", mesh_ply)
print("   GLB :", mesh_glb)


Points in cloud: 4988276
Mesh vertices: 2151595 triangles: 4302988
[Open3D WARNING] Write Ply clamped color value to valid range

✅  Poisson mesh saved:
   PLY : /content/drive/MyDrive/Drone3DProject/outputs/mesh_poisson_o3d.ply
   GLB : /content/drive/MyDrive/Drone3DProject/outputs/mesh_poisson_o3d.glb


In [ ]:
import open3d as o3d
import numpy as np

# Paths from your project
dense_dir = "/content/drive/MyDrive/Drone3DProject/dense"
pc_path = f"{dense_dir}/fused.ply"
mesh_path = f"/content/drive/MyDrive/Drone3DProject/outputs/mesh_poisson_o3d.ply"
output_mesh_path = f"{dense_dir}/meshed-poisson-colored.ply"

# Load the point cloud with color
pcd = o3d.io.read_point_cloud(pc_path)
if not pcd.has_colors():
    raise RuntimeError("❌ The point cloud does not have colors.")

# Load the Poisson mesh
mesh = o3d.io.read_triangle_mesh(mesh_path)
mesh.compute_vertex_normals()

# Build KD-tree of point cloud
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

# Transfer nearest neighbor color to each mesh vertex
colors = []
for vertex in mesh.vertices:
    _, idx, _ = pcd_tree.search_knn_vector_3d(vertex, 1)
    colors.append(pcd.colors[idx[0]])

mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

# Save the colored mesh
o3d.io.write_triangle_mesh(output_mesh_path, mesh, write_vertex_colors=True)
print(f"✅ Colored mesh saved at: {output_mesh_path}")

✅ Colored mesh saved at: /content/drive/MyDrive/Drone3DProject/dense/meshed-poisson-colored.ply


In [ ]:
!pip install -q open3d>=0.17.0

## 6 – Export OBJ & GLB

In [ ]:
o3d.io.write_triangle_mesh(f"{OUTPUTS_DIR}/final_model_clean.obj", mesh)
o3d.io.write_triangle_mesh(f"{OUTPUTS_DIR}/final_model_clean.glb", mesh)

NameError: name 'OUTPUTS_DIR' is not defined

## 7 – (Optional) Gaussian Splats – enable as needed

## 8 – Zip outputs and download

In [ ]:
!zip -qr "{PROJECT_DIR}/FinalOutputs.zip" "{OUTPUTS_DIR}"
files.download(f"{PROJECT_DIR}/FinalOutputs.zip")



zip error: Interrupted (aborting)


FileNotFoundError: Cannot find file: /content/drive/MyDrive/Drone3DProject/FinalOutputs.zip